In [95]:
import numpy as np
import pandas as pd
import praw
import html
import ast
import re
import ftfy
from markdown_it import MarkdownIt
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy

In [7]:
user_agent = "AskDoc Scraper"
posts_data = []


In [8]:
rd = praw.Reddit(
    client_id = "_JTrgh_onvOkRlXXuYk7xQ",
    client_secret = "yjEhrCabnwnNb8gxXaQGL17Cn2Nsag",
    user_agent = user_agent
)
subreddit = rd.subreddit('AskDocs')

In [15]:
def scrape_data(sort_type = "hot", limit = 1000):
    try:
        if sort_type not in ['hot', 'top']:
            raise ValueError("sort_type must be 'hot' or 'top'")
        posts_sort = subreddit.top(limit = limit) if sort_type == 'top' else subreddit.hot(limit = limit)
        for index, post in enumerate(posts_sort):
            if index % 10 == 0:
                print(f"Accessed {index} posts") 
                
            post_data = {
                'id': post.id,
                'title': post.title,
                'selftext': post.selftext,
                'author': post.author.name if post.author else None,
                'score': post.score,
                'num_comments': post.num_comments,
                'created_utc': post.created_utc,
                'flair': post.link_flair_text
            }
            
            comments_data = []
            post.comments.replace_more(limit=0)
            for comment in post.comments.list():
                comment_data = {
                    'id': comment.id,
                    'body': comment.body,
                    'author': comment.author.name if comment.author else None,
                    'score': comment.score,
                    'parent_id': comment.parent_id,
                    'created_utc': comment.created_utc,
                    'is_submitter': comment.is_submitter
                }
                comments_data.append(comment_data)
            
            post_data['comments'] = comments_data
            posts_data.append(post_data) 
            
            if index % 100 == 0 and index != 0:  
                initial_df = pd.DataFrame(posts_data)
                initial_df.to_csv(f'reddit_data_{index}.csv', index=False)
                print(f"Data saved at {index} posts")
                
    except Exception as e:
        print(f"An error occurred: {e}")
        initial_df = pd.DataFrame(posts_data)
        initial_df.to_csv('reddit_data_error_save.csv', index=False)
        raise
    
    posts_df = pd.DataFrame(posts_data)
    posts_df.to_csv(f'AskDoc_{sort_type}data.csv', index = False)
    print("Succesfully scraped the data")
    return posts_df


In [16]:
top_posts = scrape_data("top")


Accessed 0 posts
Accessed 10 posts
Accessed 20 posts
Accessed 30 posts
Accessed 40 posts
Accessed 50 posts
Accessed 60 posts
Accessed 70 posts
Accessed 80 posts
Accessed 90 posts
Accessed 100 posts
Data saved at 100 posts
Accessed 110 posts
Accessed 120 posts
Accessed 130 posts
Accessed 140 posts
Accessed 150 posts
Accessed 160 posts
Accessed 170 posts
Accessed 180 posts
Accessed 190 posts
Accessed 200 posts
Data saved at 200 posts
Accessed 210 posts
Accessed 220 posts
Accessed 230 posts
Accessed 240 posts
Accessed 250 posts
Accessed 260 posts
Accessed 270 posts
Accessed 280 posts
Accessed 290 posts
Accessed 300 posts
Data saved at 300 posts
Accessed 310 posts
Accessed 320 posts
Accessed 330 posts
Accessed 340 posts
Accessed 350 posts
Accessed 360 posts
Accessed 370 posts
Accessed 380 posts
Accessed 390 posts
Accessed 400 posts
Data saved at 400 posts
Accessed 410 posts
Accessed 420 posts
Accessed 430 posts
Accessed 440 posts
Accessed 450 posts
Accessed 460 posts
Accessed 470 posts
Acc

In [18]:
hot_posts = scrape_data("hot")

Accessed 0 posts
Accessed 10 posts
Accessed 20 posts
Accessed 30 posts
Accessed 40 posts
Accessed 50 posts
Accessed 60 posts
Accessed 70 posts
Accessed 80 posts
Accessed 90 posts
Accessed 100 posts
Data saved at 100 posts
Accessed 110 posts
Accessed 120 posts
Accessed 130 posts
Accessed 140 posts
Accessed 150 posts
Accessed 160 posts
Accessed 170 posts
Accessed 180 posts
Accessed 190 posts
Accessed 200 posts
Data saved at 200 posts
Accessed 210 posts
Accessed 220 posts
Accessed 230 posts
Accessed 240 posts
Accessed 250 posts
Accessed 260 posts
Accessed 270 posts
Accessed 280 posts
Accessed 290 posts
Accessed 300 posts
Data saved at 300 posts
Accessed 310 posts
Accessed 320 posts
Accessed 330 posts
Accessed 340 posts
Accessed 350 posts
Accessed 360 posts
Accessed 370 posts
Accessed 380 posts
Accessed 390 posts
Accessed 400 posts
Data saved at 400 posts
Accessed 410 posts
Accessed 420 posts
Accessed 430 posts
Accessed 440 posts
Accessed 450 posts
Accessed 460 posts
Accessed 470 posts
Acc

In [33]:
data1 = pd.read_csv('AskDoc_hot_data1.csv')
# data1_copy = data1.copy()
data2 = pd.read_csv('AskDoc_hot_data2.csv')
# data2_copy = data2.copy()
data3 = pd.read_csv('AskDoc_topdata.csv')
# data3_copy = data3.copy()

merged_data = pd.concat([data1, data2, data3], ignore_index = True)
merged_data.reset_index(drop = True, inplace = True)


final_data = merged_data.copy()

## Data Preprocessing
Notes for later:

handle null data

handle duplicates

Removing mod comments

Since hot posts may have unmoderated comment, can't just rely on reddit rules to assume the data's good. Thus, gotta use all sorts of methods to clean the posts and comments before fine tuning the llm. My plan:

replace special characters

replace html stuff

replace encoded characters

detect non english language and exclude them

spell check




In [34]:
#null data handling
print(f"Initial number of rows for the scraped data: {len(final_data)}")

Initial number of rows for the scraped data: 6956


In [35]:
mdi = MarkdownIt()

In [36]:
def clean_text(text, author):
    # Skip processing for bot authors or deleted/removed content
    if author in ('AskDocs-ModTeam', 'AutoModerator') or text in ('[removed]', '[deleted]'):
        return None 

    emojis = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    
    # Continue processing if not by a bot or deleted/removed
    text = mdi.render(text)
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = html.unescape(text)
    text = text.lower()  # Normalize case
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = emojis.sub(r'', text)
    
    return text

final_data['title'] = final_data.apply(lambda row: clean_text(row['title'], row['author']), axis=1)
final_data['selftext'] = final_data.apply(lambda row: clean_text(row['selftext'], row['author']), axis=1)


In [37]:
def clean_comments(comments, author):
    comments_list = ast.literal_eval(comments)
    cleaned_comments = []
    for comment in comments_list:
        if comment['author'] not in ('AskDocs-ModTeam', 'AutoModerator') and comment['body'] not in ('[removed]', '[deleted]'):
            cleaned_text = clean_text(comment['body'], comment['author'])
            if cleaned_text is not None:
                cleaned_comments.append(cleaned_text)
    return cleaned_comments

final_data['comments'] = final_data.apply(lambda row: clean_comments(row['comments'], row['author']), axis=1)

In [45]:
new_data = final_data.dropna()
# final_data = final_data[final_data['comments'].str.len() > 5]
print(f"Number of rows after dropping null values: {len(new_data)}")

Number of rows after dropping null values: 4777


In [39]:
# new_data.to_csv('CleanData.csv', encoding = 'utf-8', index = False)
# new_data.head(50)

In [47]:
# new_data = pd.read_csv('CleanData.csv')
# new_data.head(3)
for x in new_data['comments']:
    print(x[0])
    break

sorry this is happening you dont say your moms age but its unusual to develop schizophrenia later in life there are many other things that can cause auditory and visual hallucinations that arent schizophrenia some of which are more likely than schizophrenia depending on her age although having a sibling with it does increase her risk alcohol use disorder can predispose someone to certain types of brain dysfunction and dementia that could look like this too its also worth noting that a ct scan cannot rule in or out schizophrenia its hard to speculate further based on the information you have


In [51]:
def standardize_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()

    #Stop words
    stop_words = set(stopwords.words('english'))
    
    # Lemmatize tokens and remove stopwords and non-alphabetic tokens
    lemmatized_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token.isalpha()
    ]
    
    # Re-join lemmatized tokens into a single string
    return ' '.join(lemmatized_tokens)


In [52]:
stand_data = new_data.copy()

In [54]:
stand_data['title'] = stand_data['title'].apply(standardize_text)

In [55]:
stand_data['selftext'] = stand_data['selftext'].apply(standardize_text)

In [56]:
stand_data['comments'] = stand_data['comments'].apply(lambda x: [standardize_text(comment) for comment in x])

In [89]:
# final_data.drop(columns = ['author', 'num_comments', 'flair', 'id'])
stand_data.drop(columns = ['author', 'num_comments', 'flair', 'id'], inplace = True)
# stand_data.to_csv('StandardizedData.csv')

# stand_data['comment'] = stand_data['comments'].apply(lambda x: x[0] if x else '')
stand_data.head(10)



,title,selftext,score,created_utc,comments
1,mother hospitalized with acute psychosis is it...,i wa informed that my mother and best friend w...,60,1.713978e+09,[sorry this is happening you dont say your mom...
3,what are the actual chance of this happening,my son wa stillborn last month when i wa week ...,425,1.713915e+09,[sorry for your loss my deepest sympathy true ...
4,is it weird or unsafe to clean your butthole i...,age sex male height weight race hispanic durat...,228,1.713921e+09,[short answer this is kinda strange but youre ...
5,seeking adviceavenues to explore leg paralysis...,month ago i woke up with extreme leg pain to t...,9,1.713980e+09,[usual disclaimer no one can provide specific ...
7,ultrasound report,male lump on right testicle complain about inc...,5,1.713994e+09,[hello my friend im wary of firing off anythin...
8,light green pee advice pls,i am a year old female i workout and i smoke c...,3,1.713994e+09,[hello there this is from healthlinecom blue o...
16,is anal sex safe for u,i and wife are interested in exploring anal pl...,2,1.713998e+09,[there a risk of her prolapse returning probab...
17,mastoiditis or just ingrown hair,my boyfriend is sick and he is male and ha flu...,3,1.713990e+09,[mastoiditis until proven otherwise that look ...
19,is this something my doctor shouldve told me a...,foot inch pound after reading through the resu...,2,1.713996e+09,[i dont think you should be mad about this the...
23,what else can i do,im year old male no med or smoking about pound...,2,1.713994e+09,"[consider hydrocolloid patch, day before, , , ..."


## Data analysis


In [101]:
# def sentiment_labels(text):
#     sentiment = TextBlob(text).sentiment
#     return 'Urgent' if sentiment.polarity < -0.3 else 'Non-Urgent'

# # Example usage on selftext
# stand_data['urgency'] = stand_data['selftext'].apply(sentiment_labels)
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

In [ ]:
def assign_label(row):
    # Prioritize urgent sentiment
    if row['urgency'] == 'Urgent':
        return 'Urgent Medical Advice Needed'
    # Check for common medical conditions and symptoms
    if any(term in ['fever', 'cough', 'pain'] for term in row['medical_terms']):
        return 'Specific Medical Condition'
    return 'General Medical Inquiry'

df['label'] = df.apply(assign_label, axis=1)